In [ ]:
## Train for 50 epochs. First use only fine tuning on last layer
## Then compare with fine-tuning entire network
## TODO: 
## 0. Pre-process images
##      Resize images
##      Other pre-processing
## 0.1. Check image colors
## 1. Get mean and std of dataset - done
## 2. Train on Resnet-34
## 3. Train on efficient net b3
## 4. Train on mobilenet v3 large
## 5. Write a script to plot loss + accuracy graph
## 6. Get FLOPs
## 7. Get num layers
## 8. Add class weights - done
## 9. Implement gradcam

In [1]:
import sys
sys.path.append('../src/')

In [2]:
%load_ext autoreload
%autoreload 2

from multilabel.train import train_model
from model import initialize_model
from utils import set_requires_grad, save_model
from multilabel.data import load_data
from plotting import plot_data_loader
from multilabel.eval import eval_model

In [3]:
import os
import copy
import time
import random
import pickle

import numpy as np
import pandas as pd
from PIL import Image
from pathlib import Path
from tqdm import tqdm

from sklearn.metrics import f1_score, confusion_matrix
from numpy.random import shuffle
import matplotlib.pyplot as plt

import torch
import torchvision
from torchvision.io import read_image
import torchvision.transforms as T
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
import torch.autograd.profiler as tprofiler
import torch.utils.data as td

plt.rcParams["savefig.bbox"] = 'tight'

In [4]:
seed = 42

random.seed(seed)

# pytorch RNGs
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

# numpy RNG
np.random.seed(seed)

In [5]:
data_dir = "../data"
images_dir = "../data/processed"

In [6]:
# Get best num_workers
# for i in range(97):
#     start = time.time()
#     data_loader = load_data(images_dir,
#                                                                    batch_size = 96, 
#                                                                    input_size = 299, 
#                                                                    norm_arr = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
#                                                                    num_workers = i)
#     iter(data_loader['train']).next()[0].shape
#     print(f"{i}: {time.time()-start}")

In [7]:
# Check if GPU is available.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# Models options: resnet50, resnet34, inceptionv3, vgg16, mobile_net_v3_large, efficient_net_b1, efficient_net_b0, efficient_net_b3.
model_name = "efficient_net_b3"

# Number of classes.
num_classes = 20

# Batch Size.
batch_size = 32

# Epochs to train for.
num_epochs = 10

# Number of workers for data loader.
num_workers = 12

# Imagenet norm array passed as default value.
# norm_arr=([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
# Chest x-ray8 training dataset metrics 
norm_arr=((0.5989, 0.5510, 0.5175), (0.3358, 0.3330, 0.3377))

# Feature extract flag: False - Tune the whole model,
#                       True - Update only the reshaped layer parameters.
feature_extract = True

# Use pretrained flag: None - Use random weights
#                      String - Use pretrained weights given by String
use_pretrained = models.EfficientNet_B3_Weights.IMAGENET1K_V1

# Initialize the model for this run.
model_pyt, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=use_pretrained)

# lr start and end points for training.
lr_start = 0.01
lr_end = 0.001

# Print the model we just instantiated
#print(model_ft)

# Positive class weights.
pos_weight=torch.as_tensor([ 2.3509,  6.4782, 76.3264, 23.6350, 46.7897, 18.5694, 14.4224, 24.4805,
         1.1795,  0.6394,  9.1227, 80.2774,  0.8743,  2.1217,  7.2973, 87.3730,
        12.8151, 23.7444, 28.1492, 29.6749], dtype=torch.float)
pos_weight = pos_weight.to(device)

cuda:0


In [8]:
data_loaders = load_data(images_dir,
                         batch_size = batch_size, 
                         input_size = input_size, 
                         norm_arr = norm_arr,
                         num_workers = num_workers)

In [9]:
data_loaders['train'].dataset.classes

['Active',
 'Alert',
 'Amazed',
 'Amused',
 'Calm',
 'Cheerful',
 'Confident',
 'Conscious',
 'Creative',
 'Eager',
 'Educated',
 'Emotional',
 'Fashionable',
 'Feminine',
 'Inspired',
 'Loving',
 'Manly',
 'Persuaded',
 'Thrifty',
 'Youthful']

In [10]:
# plot_data_loader(data_loaders['train'], (2,2))

plot_data_loader(data_loader['train'], (2,2))

plot_data_loader(data_loader['test'], (2,2))

plot_data_loader(data_loader['val'], (2,2))

In [11]:
#model_pyt = torch.compile(model_pyt)
# Send model to GPU
model_pyt = model_pyt.to(device)

# Find parameters to be updated in this run.
# parameters with requires_grad = True.
params_to_update = model_pyt.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_pyt.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_pyt.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

Params to learn:
	 classifier.1.weight
	 classifier.1.bias


In [12]:
# 17 min 1 epoch - 128 batch size - inception
# Efficientnet b0 - batch 96 - epoch 50 - num_workers 2 - flip, auto cont, sharp - 

In [13]:
# Observe that all parameters are being optimized
optimizer = optim.Adam(params_to_update, lr=lr_start)

# Learning rate scheduler.
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=lr_end,
                                                 last_epoch=-1)


# Setup the loss fxn
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

# Train and evaluate
model_pyt, prof, val_history, train_history = train_model(device, model_pyt, data_loaders, 
                                                            optimizer, scheduler,
                                                            criterion, 
                                                            num_epochs=num_epochs,
                                                            num_classes=num_classes,
                                                            is_inception=(model_name=="inceptionv3"),
                                                            profiler=False)

Epoch 1/10
----------
Epoch [1/10], Step [100/348], Loss: 0.7574, Accuracy: 83.44%
Epoch [1/10], Step [200/348], Loss: 0.7917, Accuracy: 86.72%
Epoch [1/10], Step [300/348], Loss: 0.7396, Accuracy: 85.94%
train Loss: 0.9514 Acc: 84.04%
Epoch [1/10], Step [100/348], Loss: 0.6606, Accuracy: 82.19%
val Loss: 0.8480 Acc: 80.47%

Epoch 2/10
----------
Epoch [2/10], Step [100/348], Loss: 0.5354, Accuracy: 80.31%
Epoch [2/10], Step [200/348], Loss: 0.8962, Accuracy: 83.59%
Epoch [2/10], Step [300/348], Loss: 1.0052, Accuracy: 83.12%
train Loss: 0.7994 Acc: 84.59%
Epoch [2/10], Step [100/348], Loss: 0.6987, Accuracy: 85.31%
val Loss: 0.8680 Acc: 83.71%

Epoch 3/10
----------
Epoch [3/10], Step [100/348], Loss: 0.5917, Accuracy: 86.56%
Epoch [3/10], Step [200/348], Loss: 0.9824, Accuracy: 83.91%
Epoch [3/10], Step [300/348], Loss: 0.7653, Accuracy: 80.62%
train Loss: 0.7882 Acc: 84.66%
Epoch [3/10], Step [100/348], Loss: 0.5442, Accuracy: 85.94%
val Loss: 0.9042 Acc: 83.50%

Epoch 4/10
--------

In [19]:
dataset_name = 'ads'

In [20]:
save_model(model_pyt.state_dict(), '../models/', 
           f'{model_name}_{dataset_name}_{num_epochs}_{batch_size}_{lr_start}_{lr_end}_model_weights.pth')

True

In [21]:
eval_metrics = eval_model(device=device, model=model_pyt, test_loader=data_loaders['test'], is_inception=(model_name=="inceptionv3"), num_classes=7)

In [22]:
for i, v in eval_metrics.items():
    print(f"{i}: {v}")

acc: 2.294411945812808
f1: 0.19441178208659088
cm: [[[2841  358]
  [ 309  204]]

 [[3147  336]
  [ 184   45]]

 [[2797  890]
  [  13   12]]

 [[2853  799]
  [  29   31]]

 [[2915  753]
  [  24   20]]

 [[3123  504]
  [  59   26]]

 [[2536 1057]
  [  77   42]]

 [[3143  498]
  [  54   17]]

 [[2665  262]
  [ 624  161]]

 [[2646  115]
  [ 538  413]]

 [[2986  569]
  [  84   73]]

 [[2923  768]
  [  11   10]]

 [[2797   94]
  [ 479  342]]

 [[2999  211]
  [ 191  311]]

 [[3042  453]
  [ 118   99]]

 [[2881  810]
  [   8   13]]

 [[3064  519]
  [  46   83]]

 [[2738  886]
  [  46   42]]

 [[3255  398]
  [  19   40]]

 [[2236 1420]
  [   9   47]]]
outputs: [[0. 0. 1. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 1. 0. 1.]
 ...
 [0. 0. 0. ... 0. 1. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
targets: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [23]:
with open(f'../models/val_history_{model_name}_{dataset_name}_{num_epochs}_{batch_size}_{lr_start}_{lr_end}_.pickle', 'wb') as handle:
    pickle.dump(val_history, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(f'../models/train_history_{model_name}_{dataset_name}_{num_epochs}_{batch_size}_{lr_start}_{lr_end}_.pickle', 'wb') as handle:
    pickle.dump(train_history, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(f'../models/eval_metrics_{model_name}_{dataset_name}_{num_epochs}_{batch_size}_{lr_start}_{lr_end}_.pickle', 'wb') as handle:
    pickle.dump(eval_metrics, handle, protocol=pickle.HIGHEST_PROTOCOL)

#with open('filename.pickle', 'rb') as handle:
#    b = pickle.load(handle)

In [ ]:
#print(prof.key_averages(group_by_stack_n=5).table(sort_by='self_cpu_time_total', row_limit=50))

In [9]:
# Plot the training curves of validation accuracy vs. number
#  of training epochs for the transfer learning method and
#  the model trained from scratch
# vhist = []
# vhist = [h.cpu().numpy() for h in val_acc_history]
# thist = []
# thist = [h.cpu().numpy() for h in train_acc_history]

# plt.title("Accuracy vs. Number of Training Epochs")
# plt.xlabel("Training Epochs")
# plt.ylabel("Accuracy")
# #plt.plot(range(1,num_epochs+1),ohist,label="Pretrained")
# plt.plot(range(1,num_epochs+1),vhist,label="Validation")
# plt.plot(range(1,num_epochs+1),thist,label="Training")
# plt.ylim((0,1.))
# plt.xticks(np.arange(1, num_epochs+1, 1.0))
# plt.legend()
# plt.show()

In [8]:
# Plot the training curves of validation accuracy vs. number
#  of training epochs for the transfer learning method and
#  the model trained from scratch
# vhist = []
# vhist = [h for h in val_loss_history]
# thist = []
# thist = [h for h in train_loss_history]

# plt.title("Loss vs. Number of Training Epochs")
# plt.xlabel("Training Epochs")
# plt.ylabel("Loss")
# #plt.plot(range(1,num_epochs+1),ohist,label="Pretrained")
# plt.plot(range(1,num_epochs+1),vhist,label="Validation")
# plt.plot(range(1,num_epochs+1),thist,label="Training")
# plt.ylim((0,1.))
# plt.xticks(np.arange(1, num_epochs+1, 1.0))
# plt.legend()
# plt.show()